## 1.Import libraries

In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
import numpy as np
import re
import random
import src.functions as fc

## 2. Load de data 

In [3]:
df = pd.read_csv("../input/attacks.csv",encoding = "ISO-8859-1") #To load the data csv
df.head(5) # To see the first five rows of our dataset

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [4]:
df.dtypes # To known the type of all columns of the dataset

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object

In [5]:
# How big is the dataset?
df.shape 

(25723, 24)

### 2.1. Loking for NaN 

In [6]:
#Percentage of NaN along rows
null_rows = df.isnull().sum(axis = 1).apply (lambda y: y/df.shape[1]).sort_values(ascending = False)
null_rows[:5]

# With this I have seen that there are many rows in  the data frame that do not have any data. In this way, I will eliminate 
    ## those rows that present a percentage of nulls = 1 to clean up the dataframe a bit.

12861    1.0
14368    1.0
14382    1.0
14381    1.0
14380    1.0
dtype: float64

In [7]:
# To remove this rows:
df_2 = df.dropna(axis = 0, how = "all")
df_2.shape

#The number of rows was reduced from 25723 to 8703

(8703, 24)

In [8]:
#Now I will check the percentage of NaN along columns
perc_null_col = df_2.isnull().sum().apply(lambda x: x/df_2.shape[0]).sort_values(ascending=False)

In [9]:
#Time to select a determined value for remove columns with high percentage of nulls.

perc_null_col[perc_null_col > 0.7]

#Given the high percentage of cells with NaN in these columns (Unnamed22n = 0.999885, Unnamed23 =  0.999770). In addition to 
    #these columns there are more columns with no-relevant information for the prupose of the project. 
    
df_3 = df_2.drop(['Case Number', 'Unnamed: 22', 'Unnamed: 23', 'pdf', 'href formula', 'href', 'Name','Case Number.1','Injury', 'Case Number.2', 'Investigator or Source'], axis = 1) 
df_3.head()

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal (Y/N),Time,Species,original order
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,N,18h00,White shark,6303.0
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,N,14h00 -15h00,NaN,6302.0
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48,N,07h45,NaN,6301.0
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,N,NaN,2 m shark,6300.0
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,N,NaN,"Tiger shark, 3m",6299.0


In [10]:
# to avoid problems with spaces in columns names I remove and replace the spaces with underscore 
df_3.columns =df_3.columns.str.strip() #remove all possible spaces al the start or end of the strings
df_3.columns = df_3.columns.str.replace(' ', '_') #replace the space between words in string by underscore
df_3.columns


Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Sex', 'Age', 'Fatal_(Y/N)', 'Time', 'Species', 'original_order'],
      dtype='object')

### 2.2. Now is time to clean the selected columns. 

To realice this work I will work column by column to make them as clean as possible.

###### Date column ====> 25-Jun-2018
Along this column I am interested only in the month, because of the year is represented in the next column and the day it is not important for my hypothesis

In [11]:
# With this I take only the months of all the information I have in this column

df_3['Date'] = df_3['Date'].astype(str)

#with the regex syntaxis I am looking for those part of string that starts with a capital letter and that hace 2 more letters "\w{2}" 
df_3["Date"] = df_3[df_3.columns[0]].apply(lambda x: re.findall(r'\b[A-Z]\w{2}\b', x))    

#To remove the brackets Para quitar los corchetes de los valores:
df_3['Date'] = df_3['Date'].str.get(0)


###### Fatal column 

In [12]:
df_3["Fatal_(Y/N)"].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y'],
      dtype=object)

In [13]:
# Fisrt of all put everything in capital letters and remove possible spaces
df_3["Fatal_(Y/N)"] = df_3["Fatal_(Y/N)"].str.upper()

#Remove spaces in all the values in Fatal_(Y/N) column
df_3["Fatal_(Y/N)"] = df_3["Fatal_(Y/N)"].str.strip()
df_3["Fatal_(Y/N)"].value_counts()

N          4301
Y          1389
UNKNOWN      71
M             1
2017          1
Name: Fatal_(Y/N), dtype: int64

In [14]:
#To remove all the typos in this columns I apply a injury function (explained in the Readme)
df_3["Fatal_(Y/N)"] = df_3["Fatal_(Y/N)"].apply(fc.injury)
df_3["Fatal_(Y/N)"].value_counts()    

N    4301
Y    1389
Name: Fatal_(Y/N), dtype: int64

###### Type column

In [15]:
df_3["Type"].unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [16]:
# To solve the typos in Boat value, I change all the string that contains "Boat" ("Boat\w*) for Boat
df_3["Type"]  = df_3["Type"].replace('Boat\w*', "Boat", regex = True)
df_3["Type"].value_counts()

Unprovoked      4595
Provoked         574
Invalid          547
Boat             341
Sea Disaster     239
Questionable       2
Name: Type, dtype: int64

###### Activity column

In [17]:
df_3["Activity"].unique()

array(['Paddling', 'Standing', 'Surfing', ...,
       'Crew swimming alongside their anchored ship',
       '4 men were bathing', 'Wreck of  large double sailing canoe'],
      dtype=object)

In [18]:
df_3["Activity"]= df_3["Activity"].str.lower()


df_3["Activity"] = df_3["Activity"].replace([".*fish.*",".*Fish.*", "angling" ], "Fish", regex = True)
df_3["Activity"] = df_3["Activity"].replace([".*surf.*","paddleskiing" ], "Surf", regex = True)
df_3["Activity"] = df_3["Activity"].replace([".*swim.*", "bathing", "fell into the water", "treading water", ".*float.*"], "Swim", regex = True)
df_3["Activity"] = df_3["Activity"].replace([".*diving.*",  ".*under.*", ".*film.*", "snorkeling"], "Diving", regex = True)
df_3['Activity'] = df_3["Activity"].replace(['.*kayak.*','.*ship.*', '.*boat.*','.*board.*', 'canoeing','rowing','sailing','fishing for mackerel'],'On_boat',regex=True)
df_3["Activity"] = df_3["Activity"].replace([".*shore*.", "treading for clams", "standing", "walking", ".*play*.", "wading"], "Seashore", regex = True)
df_3['Activity'] = df_3["Activity"].replace(['.{,30} shark.','shark related act.*'],'Shark_related_act.',regex=True)

rare_act = df_3["Activity"].value_counts().loc[df_3['Activity'].value_counts() < 6].index
df_3.loc[df_3['Activity'].isin(rare_act),'Activity'] = 'Rare_act.'

df_3["Activity"].unique()

array(['Rare_act.', 'Seashore', 'Surf', 'Diving', 'Swim', 'Fish',
       'Shark_related_act.', 'On_boat', nan, 'sea disaster', 'Seashoreng',
       'murder'], dtype=object)

###### Year column ====> 2018.0

In [19]:
df_3['Year'] = df_3['Year'].replace(0,np.nan) # to replace al the 0 values by NaN

###### Type column 
This column is OK

###### Country column

In [20]:
df_3['Country'].value_counts().head(20)

USA                 2229
AUSTRALIA           1338
SOUTH AFRICA         579
PAPUA NEW GUINEA     134
NEW ZEALAND          128
BRAZIL               112
BAHAMAS              109
MEXICO                89
ITALY                 71
FIJI                  62
PHILIPPINES           61
REUNION               60
NEW CALEDONIA         53
CUBA                  46
MOZAMBIQUE            45
SPAIN                 44
INDIA                 40
EGYPT                 38
JAPAN                 34
CROATIA               34
Name: Country, dtype: int64

In [21]:
# To eliminate all the typos in the dataframe:

df_3['Country'] = df_3['Country'].str.strip() # to remove space in the string (in ch value along the column)
df_3['Country'] = df_3['Country'].str.lower() # to put all lowercase words 

#replace ambiguous contries information using regex:

df_3['Country'].replace(to_replace=['.*OCEAN','.* SEA'],value=np.nan,regex=True,inplace=True) #replace by NaN everything that contains "Ocean" or "Sea"
df_3['Country'].replace(['BETWEEN PORTUGAL & INDIA','IRAN / IRAQ','EGYPT / ISRAEL','RED SEA?'],np.nan,inplace=True) # replace by NaN everything that contains "BETWEEN PORTUGAL & INDIA",'IRAN / IRAQ','EGYPT / ISRAEL','RED SEA?
df_3['Country'].replace('EQUATORIAL GUINEA / CAMEROON','EQUATORIAL GUINEA',inplace=True) # replace by NaN everything that contains "EQUATORIAL GUINEA / CAMEROON','EQUATORIAL GUINE"
df_3['Country'].replace('CEYLON (SRI LANKA)','SRI LANKA',inplace=True) # replace by NaN everything that contains "CEYLON (SRI LANKA)','SRI LANKA"
df_3['Country'].replace('UNITED ARAB EMIRATES (UAE)','UNITED ARAB EMIRATES',inplace=True) # replace by NaN everything that contains "UNITED ARAB EMIRATES (UAE)','UNITED ARAB EMIRATES"
df_3['Country'].value_counts().head(20)

usa                 2229
australia           1338
south africa         579
papua new guinea     134
new zealand          128
brazil               112
bahamas              109
mexico                90
italy                 71
fiji                  65
philippines           62
reunion               60
new caledonia         53
cuba                  46
mozambique            45
spain                 44
egypt                 40
india                 40
japan                 34
croatia               34
Name: Country, dtype: int64

In [22]:
# Aggregation of countries to continents for all those countries that have more than 50 attacks using Continents functions described in Readme
df_3["Continent"] = df_3["Country"].apply(fc.continents)
df_3.head()

'''
After the function I have created a new variable (df_3["Continent"]) that has the following categories:
Europe
Oceania
Asia
Africa
Central_America
South_America
North_America
Unknown
'''

'\nAfter the function I have created a new variable (df_3["Continent"]) that has the following categories:\nEurope\nOceania\nAsia\nAfrica\nCentral_America\nSouth_America\nNorth_America\nUnknown\n'

###### Sex column 0.34 null values
I this case I try to reemplace the NaN data for a random selectinon of sex. We have different unique values: 

- 'F'
- 'M'
- **nan**
- **'M '**
- **'lli'**
- **'N'**
- **'.'**

marked in bld the characters that I proceed to remove

In [23]:
df_3['Sex'] = df_3['Sex'].astype(str) # Convert the column in a string

#To correct typos I used a replace_sex function (explained in the readme) and apply it to all the column (apply)
df_3["Sex"] = df_3["Sex"].apply(fc.replace_sex)
df_3["Sex"].unique()

array(['F', 'M'], dtype=object)

###### Age
In this case I replace the NaN for the mean values of the mean. First of all  I try to clean all the data. We have several types of error in this column

- Ages in string format
- Ages separated by *&*, *or*, *to*, *>*
- Ages with *?*

In [24]:
df_3['Age'] = df_3['Age'].astype(str) # To work with re.findall function we need string not integer. 


pattern = "\d+"  #to remove those values with strings or simbols
df_3['Age'] = df_3['Age'].apply(lambda x: re.findall(pattern, x)) #re.findall returns lists
df_3

#but now we have some cells with multiple values. I asume that in this attack, one shark attacked many people, hence the  
    #different ages in the same cell
df_3.explode('Age') # explode : Transform each element of a list-like to a row, replicating index values.

#Now we have lists, with this what I do is convert it to a string and select the elements in position 0 of each list (each one should have a unique value)
df_3['Age'] = df_3['Age'].str.get(0) 
df_3.head(5)


,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal_(Y/N),Time,Species,original_order,Continent
0,Jun,2018.0,Boat,usa,California,"Oceanside, San Diego County",Rare_act.,F,57,N,18h00,White shark,6303.0,North_America
1,Jun,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Seashore,F,11,N,14h00 -15h00,NaN,6302.0,North_America
2,Jun,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surf,M,48,N,07h45,NaN,6301.0,North_America
3,Jun,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surf,M,NaN,N,NaN,2 m shark,6300.0,Oceania
4,Jun,2018.0,Provoked,mexico,Colima,La Ticla,Diving,M,NaN,N,NaN,"Tiger shark, 3m",6299.0,Central_America


Once I have the column of clean years I process to replace the NaN. in this case, I will substitute the NaN for the mean of this column.

In [25]:
## first of all we need to convert str to int
df_3["Age"] = df_3["Age"].fillna(-1)
df_3["Age"] = df_3["Age"].astype(int)
df_3["Age"] = df_3["Age"].replace(-1, np.nan)

In [26]:
'''
Now what I'm going to do is replace all the empty data with the mean of all the ages we have.
For that, I first calculate the mean of the data that we have with the replace_mean_age function (explaining in readme) and 
then assign it to a variable ("value_to_replace") that I use to replace the NaN with the mean values (last line in this cell)
'''
value_to_replace = fc.replace_mean_age(df_3['Age'])
value_to_replace
df_3["Age"] = df_3["Age"].fillna(value_to_replace)
df_3.head(5)

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal_(Y/N),Time,Species,original_order,Continent
0,Jun,2018.0,Boat,usa,California,"Oceanside, San Diego County",Rare_act.,F,57.00,N,18h00,White shark,6303.0,North_America
1,Jun,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Seashore,F,11.00,N,14h00 -15h00,NaN,6302.0,North_America
2,Jun,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surf,M,48.00,N,07h45,NaN,6301.0,North_America
3,Jun,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surf,M,27.32,N,NaN,2 m shark,6300.0,Oceania
4,Jun,2018.0,Provoked,mexico,Colima,La Ticla,Diving,M,27.32,N,NaN,"Tiger shark, 3m",6299.0,Central_America


###### Time

In [27]:
df_3['Time'] = df_3['Time'].str.strip() #Remove spacs between string
df_3['Time'] = df_3['Time'].str.lower() #All strings in lowercase
df_3.loc[df_3['Time'].notnull(),'Time'].sample(20) #Return a boolean same-sized object indicating if the values are not NA.

2566             20h00
4920             17h30
1654             19h30
2645             21h30
1101             10h00
4928             13h00
5142         afternoon
3985             11h00
1220              dusk
429              09h00
1684             09h00
3127             03h00
2487             18h12
301           midnight
444              10h20
3415         afternoon
4049    10h30 or 13h30
2003             09h00
102              13h30
889              15h30
Name: Time, dtype: object

In [28]:
df_3['Time'].replace(['.*evening.*','.*dusk.*',
                         '.*dark.*','.*night.*'],'night',regex=True,inplace=True)

df_3['Time'].replace(['pm','p\.m\.','.*midday.*','.*sunset.*','.*lunchtime.*',
                         '.*noon.*','.*daytime.*','.*afternoon.*'],'afternoon',regex=True,inplace=True)

df_3['Time'].replace(['a\.m\.''am','.*dawn.*',
                         '.*daybreak.*','.*morning.*'],'day-morning',regex=True,inplace=True)

In [29]:
#grouping 06:00 to 11:59 as day-morning because english is weird
df_3['Time'].replace(to_replace='.*0[6-9].?\d{2}.*',value='day-morning',regex=True,inplace=True)
df_3['Time'].replace(to_replace='.*1[0-1].?\d{2}.*',value='day-morning',regex=True,inplace=True)

#grouping 12:00 to 17:59 as afternoon
df_3['Time'].replace(to_replace='.*1[2-7].?\d{2}.*',value='afternoon',regex=True,inplace=True)

#grouping 18:00 to 23:59 as night
df_3['Time'].replace(to_replace='.*1[8-9].?\d{2}.*',value='night',regex=True,inplace=True)
df_3['Time'].replace(to_replace='.*2[0-3].?\d{2}.*',value='night',regex=True,inplace=True)

#grouping 00:00 to 05:59 as night-morning because english is weird
df_3['Time'].replace(to_replace='.*0[0-5].?\d{2}.*',value='night-morning',regex=True,inplace=True)

In [30]:
#All the data that is not in the groups that I created before put it as NaN
df_3.loc[~df_3['Time'].isin(['day-morning','night-morning','night','afternoon']),'Time'] = np.nan 

In [31]:
df_3['Time'].value_counts(dropna=False)

NaN              5787
afternoon        1593
day-morning       923
night             359
night-morning      41
Name: Time, dtype: int64

###### Species

In [32]:
df_3['Species'] = df_3['Species'].str.lower()

In [33]:
'''
The species column was very dirty. To try to simplify it, I have selected 5 of the most abundant species and I have created a 
function (species, with regex) to group the data (White shark, Tiger Shark, Grey Shark, Lemon shark, Bull Shark and Unespecific)
''' 

df_3["Species"] = df_3["Species"].apply(fc.species)
df_3.head(5)

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal_(Y/N),Time,Species,original_order,Continent
0,Jun,2018.0,Boat,usa,California,"Oceanside, San Diego County",Rare_act.,F,57.00,N,night,White shark,6303.0,North_America
1,Jun,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Seashore,F,11.00,N,afternoon,Unespecific,6302.0,North_America
2,Jun,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surf,M,48.00,N,day-morning,Unespecific,6301.0,North_America
3,Jun,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surf,M,27.32,N,NaN,Unespecific,6300.0,Oceania
4,Jun,2018.0,Provoked,mexico,Colima,La Ticla,Diving,M,27.32,N,NaN,Tiger shark,6299.0,Central_America


In [34]:
#to save the cleaned dataset in new csv
df_3.to_csv("Final_Dataset.csv")

In [37]:
df_3["Pruebita"] = df_3["Species"].apply(fc.species2)
df_3

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal_(Y/N),Time,Species,original_order,Continent,Pruebita
0,Jun,2018.0,Boat,usa,California,"Oceanside, San Diego County",Rare_act.,F,57.00,N,night,White shark,6303.0,North_America,White shark
1,Jun,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Seashore,F,11.00,N,afternoon,Unespecific,6302.0,North_America,Unespecific
2,Jun,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surf,M,48.00,N,day-morning,Unespecific,6301.0,North_America,Unespecific
3,Jun,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surf,M,27.32,N,NaN,Unespecific,6300.0,Oceania,Unespecific
4,Jun,2018.0,Provoked,mexico,Colima,La Ticla,Diving,M,27.32,N,NaN,Tiger shark,6299.0,Central_America,Tiger shark
5,Jun,2018.0,Unprovoked,australia,New South Wales,"Flat Rock, Ballina",Surf,M,27.32,N,NaN,Unespecific,6298.0,Oceania,Unespecific
6,Jun,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",Swim,M,18.00,Y,afternoon,Tiger shark,6297.0,South_America,Tiger shark
7,May,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fish,M,52.00,N,NaN,Lemon shark,6296.0,North_America,Lemon shark
8,May,2018.0,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",Seashore,M,15.00,N,afternoon,Lemon shark,6295.0,North_America,Lemon shark
9,May,2018.0,Unprovoked,usa,Florida,"Daytona Beach, Volusia County",Seashore,M,12.00,N,afternoon,Unespecific,6294.0,North_America,Unespecific
